In [11]:
pip install pygmo



SyntaxError: invalid syntax (1132568221.py, line 1)

In [ ]:
import math
import pygmo
from scipy.special import gamma
import numpy as np
from scipy.special import gamma
from sympy import Symbol, bernoulli
from scipy.stats import chi2_contingency
from scipy.stats import poisson
from scipy.stats import norm

In [14]:

def levy_flight(z, m, c):
    """
    Compute the probability distribution based on the given formula.
    
    Parameters:
        z (float): Value of z.
        m (float): Value of m.
        c (float): Constant c.

    Returns:
        float: Probability distribution result.
    """
    step = np.zeros(dim)
    for j in range(dim):
        step[j] = (m ** (z * (m - 1))) * min(z - m, c * z - m)
    return step

In [ ]:

import numpy
import random as rnd
from numpy import random

def OptimizerwithLevy(Max_iter, SearchAgents_no, dim, lb, ub, objf, Elite_agents, Inferior_agents):
    
    # initialize of Sentinel, Elite, Inferior

    Sentinel_signal = 0

    Global_pos = numpy.zeros((1,dim))
    Global_score = numpy.ones(1)*math.inf

    Elite_pos = numpy.zeros((7,5,dim))
    Elite_score = numpy.ones((7,5))*math.inf

    Inferior_pos = numpy.zeros((8,5,dim))
    Inferior_score = numpy.ones((8,5))*math.inf


    if not isinstance(lb, list):
        lb = [lb] * dim
    if not isinstance(ub, list):
        ub = [ub] * dim

    # Initialize the positions of search agents
    Positions = numpy.zeros((SearchAgents_no, dim))
    Convergence_curve = numpy.zeros(Max_iter)
    fitness = numpy.ones((15))*math.inf

    # Loop counter
    print('optimizer is optimizing  "' + objf + '"')

    for i in range(0, SearchAgents_no):
        for j in range(dim):
            Positions[i, j] =(ub[j]-lb[j]) * random.random() + lb[j]

        # Calculate objective function for each search agent
        fitness[i] = eval(objf)(Positions[i, :])

    #Sorting
    Positions = sorted(Positions, key = eval(objf))
    fitness.sort()

    Global_score = fitness[0]
    Global_pos = Positions[0].copy()

    for i in range(0, len(Elite_score)):
        Elite_score[i,0] = fitness[i] 
        Elite_pos[i,0] = Positions[i].copy()

    for i in range(0, len(Inferior_score)):
        Inferior_score[i,0] = fitness[7+1]
        Inferior_pos[i,0] = Positions[7+1].copy()

    
    for it in range (0, Max_iter):
        
        a = 1*(1 - it/Max_iter)

        if it == 0 :
            lo = 1
        else :
            lo = 0 

        # Population Update
        for comm in range (lo, 5):

            for i in range(0, len(Elite_score)):
            
                for j in range(dim):
                    # E position update
                    z_value = 0.1
                    m_value = 0.1
                    c_value = 0.1
                    step = levy_flight(z_value, m_value, c_value)[0]
                    Elite_pos[i,comm,j] = Positions[i][j] - rnd.random()*step
                Elite_score[i,comm] = eval(objf)(Elite_pos[i,comm, :])
            
            for i in range(0, len(Inferior_score)):
                    # Inferior position update
                    
                for j in range(dim):

                    if rnd.random() > 0.5 : 
                        Inferior_pos[i,comm,j] = (rnd.random()* (ub[j] - lb[j])) + lb[j]
                    else:
                        
                        u =rnd.randint(0,14)
                        Inferior_pos[i,comm,j] = Inferior_pos[i,comm,j] + a*rnd.random()*(Inferior_pos[i,comm,j]- Inferior_pos[i,comm,j])

                Inferior_score[i,comm] = eval(objf)(Inferior_pos[i,comm, :])

                if Inferior_score[i,comm] < Global_score :

                    # Signal for regrouping
                    Sentinel_signal = 1
                    # Perform regrouping

                    # get min for every agent and sent that value to position and fitness

                    Int_score =  numpy.concatenate((Elite_score, Inferior_score), axis = 0)
                    Int_pos  =  numpy.concatenate((Elite_pos, Inferior_pos), axis = 0)
                    Bets = Int_score.min(axis = 1)

                    for i in range(0, SearchAgents_no):
                        array4 = list(Int_score[i])
                        t = array4.index(Bets[i])
                        fitness[i] = Int_score[i,t]
                        Positions[i] = Int_pos[i,t,:]

                    #Sorting
                    Positions = sorted(Positions, key = eval(objf)) # Personal Bests
                    fitness.sort()

                    if fitness[0] < Global_score :

                        Global_score = fitness[0]
                        Global_pos = Positions[0].copy()

                    for i in range(0, len(Elite_score)):

                        Elite_score[i,0] = fitness[i] 
                        Elite_pos[i,0] = Positions[i].copy()

                    for i in range(0, len(Inferior_score)):

                        Inferior_score[i,0] = fitness[7+i]
                        Inferior_pos[i,0] = Positions[7+i].copy()
                    
        # Position Update after 5 iter of Elite
        for i in range(0, len(Elite_score)):
            
            for j in range(dim):
                c1 = 1.49445 # cognitive 
                c2= 1.49445 # social 
                r1 = rnd.random()    # randomizations
                r2 = rnd.random()
                # E position update toward global best
                Elite_pos[i,comm,j] = (  r1 * ( Positions[i][j]- Elite_pos[i][4][j])) + (  r2 * (Global_pos[j]-Elite_pos[i][4][j]))
            Elite_score[i,comm] = eval(objf)(Elite_pos[i,comm, :])

        #Sorting
        Int_score =  numpy.concatenate((Elite_score, Inferior_score), axis = 0)
        Int_pos  =  numpy.concatenate((Elite_pos, Inferior_pos), axis = 0)
        Bets = Int_score.min(axis = 1)

        for i in range(0, SearchAgents_no):
            array4 = list(Int_score[i])
            t = array4.index(Bets[i])
            fitness[i] = Int_score[i,t]
            Positions[i] = Int_pos[i,t,:]

        Positions = sorted(Positions, key = eval(objf))
        fitness.sort()
        
        if fitness[0] < Global_score :

            Global_score = fitness[0]
            Global_pos = Positions[0].copy()

        for i in range(0, len(Elite_score)):
            Elite_score[i,0] = fitness[i] 
            Elite_pos[i,0] = Positions[i].copy()

        for i in range(0, len(Inferior_score)):
            Inferior_score[i,0] = fitness[7+i]
            Inferior_pos[i,0] = Positions[7+i].copy()

        Convergence_curve[it] = Global_score

        if it % 1 == 0:
            print(["At iteration " + str(it*5) + " the best fitness is " + str(Global_score)])
            
    return Global_score, Convergence_curve


In [7]:
cec_functions = {
    'F1': {'lb': -100, 'ub': 100, 'dim': 30},
    'F2': {'lb': -100, 'ub': 100, 'dim': 30},
    'F3': {'lb': -100, 'ub': 100, 'dim': 30},
    'F4': {'lb': -100, 'ub': 100, 'dim': 30},
    'F5': {'lb': -100, 'ub': 100, 'dim': 30},
    'F6': {'lb': -100, 'ub': 100, 'dim': 30},
    'F7': {'lb': -100, 'ub': 100, 'dim': 30},
    'F8': {'lb': -100, 'ub': 100, 'dim': 30},
    'F9': {'lb': -100, 'ub': 100, 'dim': 30},
    'F10': {'lb': -100, 'ub': 100, 'dim': 30},
    'F11': {'lb': -100, 'ub': 100, 'dim': 30},
    'F12': {'lb': -100, 'ub': 100, 'dim': 30},
    'F13': {'lb': -100, 'ub': 100, 'dim': 30},
    'F14': {'lb': -100, 'ub': 100, 'dim': 30},
    'F15': {'lb': -100, 'ub': 100, 'dim': 30},
    'F16': {'lb': -100, 'ub': 100, 'dim': 30},
    'F17': {'lb': -100, 'ub': 100, 'dim': 30},
    'F18': {'lb': -100, 'ub': 100, 'dim': 30},
    'F19': {'lb': -100, 'ub': 100, 'dim': 30},
    'F20': {'lb': -100, 'ub': 100, 'dim': 30},
    'F21': {'lb': -100, 'ub': 100, 'dim': 30},
    'F22': {'lb': -100, 'ub': 100, 'dim': 30},
    'F23': {'lb': -100, 'ub': 100, 'dim': 30},
    'F24': {'lb': -100, 'ub': 100, 'dim': 30},
    'F25': {'lb': -100, 'ub': 100, 'dim': 30},
    'F26': {'lb': -100, 'ub': 100, 'dim': 30},
    'F27': {'lb': -100, 'ub': 100, 'dim': 30},
    'F28': {'lb': -100, 'ub': 100, 'dim': 30},
}


In [8]:
# Driver
import pygmo
num_particles = 15
max_iter = 100
runs = 1

sso_levy = np.ones((22,runs))
Curve_sso_levy = np.zeros((22,runs,max_iter))

Elite_agents = 7
Inferior_agents = num_particles - Elite_agents

cec_function_names = {
    1: 'F1',
    2: 'F2',
    3: 'F3',
    4: 'F4',
    5: 'F5',
    6: 'F6',
    7: 'F7',
    8: 'F8',
    9: 'F9',
    10: 'F10',
    11: 'F11',
    12: 'F12',
    13: 'F13',
    14: 'F14',
    15: 'F15',
    16: 'F16',
    17: 'F17',
    18: 'F18',
    19: 'F19',
    20: 'F20',
    21: 'F21',
    22: 'F22',
    23: 'F23',
    24: 'F24',
    25: 'F25',
    26: 'F26',
    27: 'F27',
    28: 'F28'
}

for i in range(1, len(cec_function_names) + 1):
    for j in range(runs):
        func = cec_function_names[i]
        lol = cec_functions.get(func, {})
        lb = lol.get('lb')
        ub = lol.get('ub')
        dim = lol.get('dim')

        print(f"{func} function, dim = {dim}")

        sso_levy[i-1, j], Curve_sso_levy[i-1, j, :] = OptimizerwithLevy(max_iter, num_particles, dim, lb, ub, func,  Elite_agents, Inferior_agents)

    

ModuleNotFoundError: No module named 'pygmo'

In [ ]:
pip install pygmo

